In [1]:
import notebooks_setup

notebooks_setup.setup()

Added C:\Users\abawa012\Downloads\repos\cluster-links to Python path.
Added C:\Users\abawa012\Downloads\repos\cluster-links\src to Python path
Current working directory: c:\Users\abawa012\Downloads\repos\cluster-links\notebooks


In [2]:
column_mappings = {
    "window": "folder",
    "url": "href",
    "hostname": "hostname",
    "title": "title",
}

In [3]:
import pandas as pd

csv_infile = notebooks_setup.DATA_INPUT_DIR.glob("*.csv").__next__()
json_infile = notebooks_setup.DATA_INPUT_DIR.glob("*.json").__next__()

df = pd.read_csv(csv_infile).rename(columns=column_mappings).fillna("")

In [4]:
df["folder"] = df["folder"].str.extract(r"([a-zA-Z\-]+[a-zA-Z])")

In [5]:
def _dedup_href(group: pd.DataFrame) -> pd.DataFrame:
    if len(group) == 1:
        return group

    group_columns = group.columns
    max_record = group[group_columns[0]].str.len()
    for i in range(1, len(group_columns)):
        max_record += group[group_columns[i]].str.len()
    max_idx = max_record.idxmax()

    return group.loc[[max_idx]]

In [6]:
df = (
    df.drop_duplicates()
    .loc[df["href"].str.strip() != "chrome://newtab/"]
    .groupby("href")
    .apply(_dedup_href, include_groups=False)
    .reset_index()  # [list(column_mappings.values())]
)

In [8]:
bm_folder_list = list(
    map(
        lambda x: {
            "title": x,
            "links": df.loc[df["folder"] == x][
                [c for c in column_mappings.values() if c != "folder"]
            ].to_dict(orient="records"),
        },
        df["folder"].unique().tolist(),
    )
)
bm_root = {"title": "Unnamed", "children": bm_folder_list}

In [9]:
from data_collection.models import load_bookmarks_from_dict

bookmarks = load_bookmarks_from_dict(bm_root)
# Get statistics
print(f"Total bookmarks: {bookmarks.total_bookmarks}")
print(f"Total folders: {bookmarks.total_folders}")

Total bookmarks: 2072
Total folders: 37


In [11]:
# Find bookmarks by domain
stanford_bookmarks = []
for folder in bookmarks.children:
    stanford_bookmarks.extend(folder.find_links_by_domain("stanford.edu"))
list(map(lambda x: (x.domain, x.title, x.href), stanford_bookmarks))[:5]

[('cs224d.stanford.edu',
  'Stanford University CS224d: Deep Learning for Nat…',
  'http://cs224d.stanford.edu/'),
 ('cs231n.stanford.edu',
  'Stanford University CS231n: Deep Learning for Com…',
  'http://cs231n.stanford.edu/'),
 ('stanford.edu',
  'Teaching - CS 230',
  'https://stanford.edu/~shervine/teaching/cs-230/'),
 ('web.stanford.edu',
  'Stanford CS 25 | Transformers United',
  'https://web.stanford.edu/class/cs25/'),
 ('explorecourses.stanford.edu',
  'Stanford University Explore Courses',
  'https://explorecourses.stanford.edu/search?view=catalog&filter-coursestatus-Active=on&page=0&q=ME218')]